In [1]:
import numpy as np
import pandas as pd
import json
import zipfile
from bs4 import BeautifulSoup
import os
from collections import Counter

In [2]:
thing = open('data/116/bills/s/s1/data.json')
bill = json.load(thing)

In [26]:
bill.keys()

dict_keys(['actions', 'amendments', 'bill_id', 'bill_type', 'by_request', 'committee_reports', 'committees', 'congress', 'cosponsors', 'enacted_as', 'history', 'introduced_at', 'number', 'official_title', 'popular_title', 'related_bills', 'short_title', 'sponsor', 'status', 'status_at', 'subjects', 'subjects_top_term', 'summary', 'titles', 'updated_at', 'url'])

In [74]:
actions = pd.DataFrame(bill['actions'])
cosponsors = pd.DataFrame(bill['cosponsors'])
cosponsors['sponsor'] = False

In [79]:
bill['sponsor']['sponsor'] = True
sponsor = bill['sponsor']
cosponsors.append(sponsor, ignore_index = True)

In [83]:
cosponsors

,bioguide_id,district,name,original_cosponsor,sponsored_at,state,title,withdrawn_at,sponsor
0,B000575,None,"Blunt, Roy",True,2019-01-03,MO,Sen,None,False
1,C001035,None,"Collins, Susan M.",False,2019-01-14,ME,Sen,None,False
2,C001096,None,"Cramer, Kevin",False,2019-01-08,ND,Sen,None,False
3,C001098,None,"Cruz, Ted",False,2019-01-08,TX,Sen,None,False
4,G000562,None,"Gardner, Cory",True,2019-01-03,CO,Sen,None,False
5,H001079,None,"Hyde-Smith, Cindy",False,2019-01-10,MS,Sen,None,False
6,I000024,None,"Inhofe, James M.",False,2019-01-30,OK,Sen,None,False
7,M000355,None,"McConnell, Mitch",True,2019-01-03,KY,Sen,None,False
8,R000584,None,"Risch, James E.",False,2019-01-09,ID,Sen,None,False
9,R000307,None,"Roberts, Pat",False,2019-01-29,KS,Sen,None,False


In [25]:
bill_type = ['s', 'sjres', 'hr', 'hjres']
folders1 = ['bills', 'bills_text']

In [30]:
rootdir = 'data/116/bills/s'
thing = []
for folder in bill_type:
    rootdir = 'data/116/bills/'+folder
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            thing.append(os.path.join(subdir, file))

In [31]:
new_thing = []
for i in thing:
    if 'data.json' in i:
        new_thing.append(i)

In [34]:
len(new_thing)

14269

In [42]:
file = open('data/116/bills_text/s/s1/text-versions/pcs/BILLS-116s1pcs/html/BILLS-116s1pcs.htm')
soup = BeautifulSoup(file, 'html.parser')

In [86]:
thing = []
for folder in bill_type:
    rootdir = 'data/116/bills_text/'+folder
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            thing.append(os.path.join(subdir, file))

In [96]:
zipped = [x for x in thing if '.zip' in x]

In [99]:
new_thing = []
for i in thing:
    new_thing.append(i.split('text-versions/'))
    
new_new_thing = []
for i in new_thing:
    try:
        second = i[1]
        new_new_thing.append(second.split('/'))
    except:
        print(i)

folders_set = set()
for i in new_new_thing:
    folders_set.add(i[0])
    
just_folders = []
for i in new_new_thing:
    just_folders.append(i[0])

c = Counter(just_folders)

['data/116/bills_text/s/s4091/.DS_Store']
['data/116/bills_text/s/s1/.DS_Store']


In [85]:
version_def = {'ih': 'Introduced in House',
 'is': 'Introduced in Senate',
 'eh': 'Engrossed (House) - version that is the official copy as passed before it is sent to the Senate',
 'es': 'Engrossed (Senate) - version that is the official copy as passed before sent to House',
 'rh': 'Reported in (House) - version reported by the committee(s) including changes if any. It is then available for floor consideration',
 'rs': 'Reported in (Senate) - version reported by the committee(s) including changes if any. It is then available for floor consideration',
 'cph': 'Considered and passed by house',
 'cps': 'Considered and passed by Senate',
 'rfh': 'Referred to House committee after being recieved from Senate',
 'rfs': 'Referred to Senate commidttee after being recieved from House', 
 'eah': 'Engrossed Amendment (House) - same as EH but often is the engrossment of an amendment which replaces the entire text of a measure',
 'eas': 'Engrossed Amendment (Senate) - same as ES but often is the engrossment of an amendment which replaces the entire text of a measure',
 'pch': 'Version placed on House calendar for consideration',
 'pcs': 'Version placed on the Senate calendar for consideration',
 'rch': 'Reference change (House) - version re-referred to different or additional committee',
 'rcs': 'Reference change (Senate) - version re-referred to different or additional committee',
 'iph': 'Indefinitely postponed',
 'ips': 'Indefinitely postponed',
 'rdh': 'recieved in House from Senate',
 'rds': 'received in Senate from House',
 'enr': 'Initial copy of a bill or joint resolution which has passed both houses in identical form'
}

In [100]:
versions = list(version_def.keys())

In [101]:
versions

['ih',
 'is',
 'eh',
 'es',
 'rh',
 'rs',
 'cph',
 'cps',
 'rfh',
 'rfs',
 'eah',
 'eas',
 'pch',
 'pcs',
 'rch',
 'rcs',
 'iph',
 'ips',
 'rdh',
 'rds',
 'enr']

In [156]:
big_dict = {'bill_num':[],
            'type':[],
            'text':[]}

In [157]:

for i in range(len(thing)):
    try:
        path_elems = thing[i].split('/')
        unzipped_folder = "BILLS-" + path_elems[1] + path_elems[4] + path_elems[6]
        html_file_path = unzipped_folder + "/html/" + unzipped_folder + '.htm'

        zf = zipfile.ZipFile(thing[i])
        file = zf.open(html_file_path)
        soup = BeautifulSoup(file, 'html.parser')
        
        big_dict['bill_num'].append(path_elems[4])
        big_dict['type'].append(path_elems[6])
        big_dict['text'].append(soup)
        
    except:
        print(thing[i])

data/116/bills_text/s/s4091/.DS_Store
data/116/bills_text/s/s4091/text-versions/.DS_Store
data/116/bills_text/s/s4091/text-versions/enr/.DS_Store
data/116/bills_text/s/s4091/text-versions/enr/BILLS-116s4091enr/mods.xml
data/116/bills_text/s/s4091/text-versions/enr/BILLS-116s4091enr/premis.xml
data/116/bills_text/s/s4091/text-versions/enr/BILLS-116s4091enr/dip.xml
data/116/bills_text/s/s4091/text-versions/enr/BILLS-116s4091enr/pdf/BILLS-116s4091enr.pdf
data/116/bills_text/s/s4091/text-versions/enr/BILLS-116s4091enr/html/BILLS-116s4091enr.htm
data/116/bills_text/s/s4091/text-versions/enr/BILLS-116s4091enr/xml/BILLS-116s4091enr.xml
data/116/bills_text/s/s4091/text-versions/enr/BILLS-116s4091enr/uslm/BILLS-116s4091enr.xml
data/116/bills_text/s/s4091/text-versions/cps/.DS_Store
data/116/bills_text/s/s1/.DS_Store
data/116/bills_text/s/s1/text-versions/.DS_Store
data/116/bills_text/s/s1/text-versions/pcs/.DS_Store
data/116/bills_text/s/s1/text-versions/pcs/BILLS-116s1pcs/mods.xml
data/116/bil

In [159]:
bill_text_df = pd.DataFrame(big_dict)

In [160]:
bill_text_df

,bill_num,type,text
0,s1379,enr,[[[<pre>\n[Congressional Bills 116th Congress]...
1,s1379,is,[[[<pre>\n[Congressional Bills 116th Congress]...
2,s1379,es,[[[<pre>\n[Congressional Bills 116th Congress]...
3,s1977,is,[[[<pre>\n[Congressional Bills 116th Congress]...
4,s4439,is,[[[<pre>\n[Congressional Bills 116th Congress]...
...,...,...,...
6029,sjres46,es,[[[<pre>\n[Congressional Bills 116th Congress]...
6030,sjres12,is,[[[<pre>\n[Congressional Bills 116th Congress]...
6031,sjres15,is,[[[<pre>\n[Congressional Bills 116th Congress]...
6032,sjres23,is,[[[<pre>\n[Congressional Bills 116th Congress]...


In [161]:
len(thing)

6054